# 08 Linear Regression

In [1]:
import torch
import torch.nn as nn
import numpy as np 
from sklearn import datasets 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

### Prepare data

In [2]:
bc = datasets.load_breast_cancer()
x, y = bc.data, bc.target
n_sample, n_feat = x.shape
n_sample, n_feat

(569, 30)

In [3]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1234)

In [4]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [5]:
x_train = torch.from_numpy(x_train.astype(np.float32))
x_test = torch.from_numpy(x_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32)).view(y_train.shape[0], 1)
y_test = torch.from_numpy(y_test.astype(np.float32)).view(y_test.shape[0], 1)

### Model

In [6]:
class LogisticRegression(nn.Module):
    def __init__(self, n_input):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input, 1)
        
    def forward(self, x):
        return torch.sigmoid(self.linear(x))

In [7]:
model = LogisticRegression(n_feat)

### Loss and optimizer

In [10]:
criteria = nn.BCELoss()
lr = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

### Training loop

In [11]:
nepochs = 100
for epoch in range(nepochs):
    y_pred = model(x_train)
    loss = criteria(y_pred, y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    if epoch % 10 == 9:
        print(f"epoch: {epoch+1}, loss: {loss.item():.6f}")
        

epoch: 10, loss: 0.600805
epoch: 20, loss: 0.489756
epoch: 30, loss: 0.420048
epoch: 40, loss: 0.372393
epoch: 50, loss: 0.337643
epoch: 60, loss: 0.311044
epoch: 70, loss: 0.289914
epoch: 80, loss: 0.272637
epoch: 90, loss: 0.258181
epoch: 100, loss: 0.245860


In [17]:
with torch.no_grad():
    y_pred = model(x_test).round()
    acc = y_pred.eq(y_test).sum() / y_test.shape[0]
    print(f"Test accuracy: {acc:.6f}")

Test accuracy: 0.912281
